In [64]:
import torch
from torch.utils.data import Dataset, DataLoader
import h5py
import os

# Custom Dataloader for our NN from .h5 database
class HDF5Data(Dataset):
    def __init__(self, path_to_h5):
        self.path_to_h5 = path_to_h5
        with h5py.File(path_to_h5, 'r') as f:
            signaldata = f['NoisySignals'][()]
            butterparams = f['ButterworthLowPass'][()]
            datalen = len(signaldata)
            
        self.signaldata = signaldata
        self.butterparams = butterparams
        self.datalen = datalen

    def __len__(self):
        return self.datalen

    def __getitem__(self, idx):
        return self.signaldata[idx], self.butterparams[idx]

In [65]:
PATH_TO_H5 = '/Users/aaronphilip/ScienceFair/projects/NanoporeSequencingFiltering/database/NanoporeFiltered.h5'
dataloader = HDF5Data(PATH_TO_H5)
train_dataloader = DataLoader(dataloader, batch_size=64, shuffle=False)

train_signal, train_params = next(iter(train_dataloader))

In [66]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device
